In [1]:
library(GEOquery)
library(limma)
library(tibble)
library(Biobase)
library(rlist)
library(tidyverse)
library(reshape2)
library(ath1121501.db)# Affymetrix Arabidopsis ATH1 Genome Array annotation data

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; vie

In [2]:
logChange <- function (expression){
    ex <- expression
    qx <- as.numeric(quantile(ex, c(0., 0.25, 0.5, 0.75, 0.99, 1.0), na.rm=T))
    LogC <- (qx[5] > 100) ||
      (qx[6]-qx[1] > 50 && qx[2] > 0) ||
      (qx[2] > 0 && qx[2] < 1 && qx[4] > 1 && qx[4] < 2)

    if (LogC) { ex[which(ex <= 0)] <- NaN
    print("log2 transform finished")}else{print("log2 transform not needed")}
    return(log2(ex))
}

getgeo <- function(aFile){
    ###get the data
    gset <- getGEO(aFile, GSEMatrix =TRUE, AnnotGPL=F )
    #get the expression set which is a list
    expressionSet <- gset[[1]]
    exprSet <- exprs(expressionSet)
    #data description
    pData <- pData(expressionSet)
    colnames(exprSet) <- sapply(pData$title,function (x) substr(x,14,100))
    exprSet <- logChange(exprSet)
    return(as.data.frame(exprSet))
}

anno_rmDup <- function (express){
    express$ID <- rownames(express)
    anno_exprSet <- merge(x=geneAnno,y=express,by='ID') # expreSet must be a dataframe for merge, need to convert to a dataframe from original exprSet
    anno_exprSet$ID <- NULL
    anno_exprSet <- anno_exprSet %>% 
                    mutate(rowMean =rowMeans(.[-1]))%>%  # . means data from last step
                    arrange(desc(rowMean)) %>% 
                    distinct(AGI, .keep_all = T) %>% 
                    dplyr::select(-rowMean)%>% # use dplyr::select to avoid conflict with R built-in function select
                    column_to_rownames(var = "AGI")
    return(anno_exprSet)
}

deg <- function(sample){ #express dataset: index are gene names; total four columns, first two columns are control, last two columns are treatment
    group <- c(rep('control',2), rep('treatment',2))
    # set the group mandatorily, or the comparison refrence will be assigned based on alphabet
#     group <- relevel(group, ref="control") 
    
    #make the design matrix and comparision
    design <- model.matrix(~0+factor(group))
    colnames(design) <- levels(factor(group))
    rownames(design) <- colnames(sample)
    contrast.matrix <- makeContrasts(contrasts = "treatment - control", levels = design)
    # paste0(unique(group),collapse = "-")
    
    # fit the model in limma
    fit <- lmFit(sample, design)
    fit1 <- contrasts.fit(fit, contrast.matrix)
    fit2 <- eBayes(fit1)
    
    dif <- topTable(fit2, coef=1, number=Inf)
    #filter the day by p.value <= 0.05
    dif <- dif[(dif$logFC >1)|(dif$logFC <-1)&(dif$P.Value <=0.05),]
    return(dif)
}

In [3]:
############################################ Annotation file ########################################################################
gpl <- getGEO('GPL198')
tgpl <- Table(gpl)
geneAnno <- tgpl[, c('ID','AGI')] 
geneAnno <- geneAnno[!(geneAnno$AGI=='' | grepl('ATMG|ATCG|/',geneAnno$AGI)),]
############################################ Prepare the dataset ###############################################################
#GSE5620:Control, GSE5621:Cold,GSE5622:Osmotisis, GSE5623:Salt, GSE5624:Drought 
#GSE5625:Genotoxic stress, GSE5626:UV-B stress,GSE5627:Wounding stress, GSE5628:Heat stress
stresslist <- paste0('GSE562',seq(0,8))
data <- list()
for (i in seq(length(stresslist))) data[[i]]=anno_rmDup(getgeo(stresslist[i])) # extract data to the list

# extract data except 0h, 0.25h and 4.0h and whole data for heat stress, and the excluded data will be analysized following 
data2 <- list()
for (i in seq(length(data)-1)) data2[[i]] <- data[[i]][,!grepl('-0h|0.25h|-4.0h',colnames(data[[i]]))] # remove 0h, 0.25h and 4.0h in control dataset
# check dataframe columns length
# for (k in seq(length(data))) print(length(colnames(data[[k]]))) 

File stored at: 

/tmp/RtmpJpJjqV/GPL198.soft

Warning message:
“64 parsing failures.
  row     col           expected    actual         file
22747 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22748 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22749 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22750 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22751 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
..... ....... .................. ......... ............
See problems(...) for more details.
”
Found 1 file(s)

GSE5620_series_matrix.txt.gz

Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)

See spec(...) for full column specifications.

Using locally cached version of GPL198 found here:
/tmp/RtmpJpJjqV/GPL198.soft 

Warning message:
“64 parsing failures.
  row     col           expected    actual         file
22747 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22748 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22749

[1] "log2 transform finished"


Found 1 file(s)

GSE5621_series_matrix.txt.gz

Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)

See spec(...) for full column specifications.

Using locally cached version of GPL198 found here:
/tmp/RtmpJpJjqV/GPL198.soft 

Warning message:
“64 parsing failures.
  row     col           expected    actual         file
22747 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22748 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22749 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22750 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22751 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
..... ....... .................. ......... ............
See problems(...) for more details.
”


[1] "log2 transform finished"


Found 1 file(s)

GSE5622_series_matrix.txt.gz

Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)

See spec(...) for full column specifications.

Using locally cached version of GPL198 found here:
/tmp/RtmpJpJjqV/GPL198.soft 

Warning message:
“64 parsing failures.
  row     col           expected    actual         file
22747 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22748 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22749 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22750 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22751 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
..... ....... .................. ......... ............
See problems(...) for more details.
”


...

[1] "log2 transform finished"


Found 1 file(s)

GSE5627_series_matrix.txt.gz

Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)

See spec(...) for full column specifications.

Using locally cached version of GPL198 found here:
/tmp/RtmpJpJjqV/GPL198.soft 

Warning message:
“64 parsing failures.
  row     col           expected    actual         file
22747 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22748 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22749 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22750 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22751 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
..... ....... .................. ......... ............
See problems(...) for more details.
”


[1] "log2 transform finished"


Found 1 file(s)

GSE5628_series_matrix.txt.gz

Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)

See spec(...) for full column specifications.

Using locally cached version of GPL198 found here:
/tmp/RtmpJpJjqV/GPL198.soft 

Warning message:
“64 parsing failures.
  row     col           expected    actual         file
22747 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22748 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22749 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22750 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
22751 SPOT_ID 1/0/T/F/TRUE/FALSE --Control literal data
..... ....... .................. ......... ............
See problems(...) for more details.
”


[1] "log2 transform finished"


In [4]:
############################################# Run the program##################################################################
count <- 1
allDEG <- c()
while (count<24){
    con <- data2[[1]] %>% mutate(AGI=rownames(.)) %>% dplyr::select(c(count,count+1,'AGI'))
    for (num in seq(2,length(data2))){
        tre <- data2[[num]] %>% mutate(AGI=rownames(.)) %>% dplyr::select(c(count,count+1,'AGI'))
        merdata <- merge(con,tre,by='AGI') %>% remove_rownames %>% column_to_rownames(var="AGI")
        allDEG <- c(allDEG,rownames(deg(merdata))) # call deg function
        allDEG <- unique(allDEG)
    }
    count <- count+2
}

In [ ]:
for (k in seq(length(data))) print(length(colnames(data[[k]]))) 

In [5]:
############################################ Extra DEG for drought, UVB, wounding, heatstress###################################
# Drought, UVB, wounding for 0.25h
extraDEG <- c()
control_0.25_shoots <- data[[1]] %>% mutate(AGI=rownames(.)) %>% dplyr::select(c(5,6,'AGI'))
control_0.25_roots <- data[[1]] %>% mutate(AGI=rownames(.)) %>% dplyr::select(c(7,8,'AGI'))

for (i in c(5,7,8)){
    treat1 <- data[[i]] %>% mutate(AGI=rownames(.)) %>% dplyr::select(c(1,2,'AGI'))
    treat2 <- data[[i]] %>% mutate(AGI=rownames(.)) %>% dplyr::select(c(3,4,'AGI'))
    merdata1 <- merge(control_0.25_shoots,treat1,by='AGI') %>% remove_rownames %>% column_to_rownames(var="AGI")
    merdata2 <- merge(control_0.25_roots,treat2,by='AGI') %>% remove_rownames %>% column_to_rownames(var="AGI")
    extraDEG <- c(extraDEG,rownames(deg(merdata1))) # call deg function
    extraDEG <- c(extraDEG,rownames(deg(merdata2))) # call deg function
    extraDEG <- unique(extraDEG)
}

# heatstress 0.25h, 0.5h 1.0h, 3.0h
control_3h <- data[[1]][,seq(5,20)]
heat_3h <- data[[9]][,seq(1,16)]
count <- 1
while (count<16){
    con <- control_3h %>% mutate(AGI=rownames(.)) %>% dplyr::select(c(count,count+1,'AGI'))
    tre <- heat_3h %>% mutate(AGI=rownames(.)) %>% dplyr::select(c(count,count+1,'AGI'))
    merdata <- merge(con,tre,by='AGI') %>% remove_rownames %>% column_to_rownames(var="AGI")
    extraDEG <- c(extraDEG,rownames(deg(merdata))) # call deg function
    extraDEG <- unique(extraDEG)

    count <- count+2
}

In [6]:
print(length(allDEG))
print(length(extraDEG))

[1] 20324
[1] 18521


In [8]:
# combine all DEG under all stresses
allStressDEG <- unique(c(allDEG,extraDEG))
print(length(allStressDEG))

[1] 20335


In [9]:
# Control genes from PEG timeseries treatment not differentially regulated
allnegative <- read.table('allNegativeGeneList.txt')
negGeneList_tmp <- allnegative$V1
# remove genes in negative gene list while not in DEG 
negGeneList <- negGeneList_tmp[!negGeneList_tmp %in% allStressDEG]

In [10]:
length(negGeneList)

[1] 4210

In [20]:
# export the negative gene list to a txt
for (gene in negGeneList){
    sink('negativeGeneList_rmStressDEG.txt',append=TRUE)
    cat(gene,'\n')
    sink()
}